In [1]:
import sys

## !pip install -e ../gym_space_docking/

loc = os.popen('pip3 show gym_space_docking').readlines()[7].split()[1]


sys.path.append(loc)


import gym
import os
import gym_space_docking
# include Network
import numpy as np
import tensorflow as tf

from tensorflow import keras
#from tensorflow.keras.layers import Embedding, Dense, Input, Dropout, LSTM, Activation, Conv2D, Reshape, Average, Bidirectional
from tensorflow.keras import layers
import collections
import PIL

In [2]:
env = gym.make('space_docking-v0')

pygame 2.1.0 (SDL 2.0.16, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
/Users/denny/Documents/GitHub/IronHack_Final_Project/gym_space_docking/gym_space_docking/envs


In [3]:

#env = gym.wrappers.Monitor(env, 'recording')

In [4]:
num_actions = env.action_space.n


def create_q_model():
    # Network defined by the Deepmind paper
    #inputs = layers.Input(shape=(88, 80, 3,))
    inputs = layers.Input(shape=(4, 88, 80,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu", padding='same')(inputs) 
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu", padding='same')(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu", padding='same')(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

In [5]:
#model = keras.models.load_model('train_model')
#model_target = keras.models.load_model('target_model')

In [6]:
model = create_q_model()
model_target = create_q_model()

2022-03-18 12:22:23.343030: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

In [8]:


# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50000 
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0 # 
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000 #
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()
#loss
loss = 0.0

# frames which get ignored
skip_frames = 4

done = False
reward = 0

file_rewards = open('rewards.txt', 'w')
file_epsilon = open('epsilon.txt', 'w')
file_loss = open('loss.txt', 'w')

# write points to files

while True:  # Run until solved
    
    obs , reward, done, _ = env.reset()
    im = np.array(PIL.Image.fromarray(obs).convert('L'))
    im = im.reshape(1, im.shape[0], im.shape[1])
    state = np.repeat(im, 4, axis=0)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state) #, dtype=tf.float32) # add dtype float 32
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        info = ''
        # Apply the sampled action in our environment

        #for i in range(skip_frames):
        #    _, _, _, _ = env.step(action)    

        obs, n_reward, n_done, info = env.step(action)

        im = np.array(PIL.Image.fromarray(obs).convert('L'))

        state_next = state.copy()

        # copy and shift old frames
        if not obs is None or not reward or not done:
            #print('debug')
            state_next[3] = state_next[2]
            state_next[2] = state_next[1]
            state_next[1] = state_next[0]
            state_next[0] = im
            reward = n_reward
            done = n_done
        else:
            print('missing obs or reward or done')


        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)

        #print(state.shape)

        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        #
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
        
            
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )


            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample) # add np.array

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            
            file_rewards.write(str(rewards_history[-1]) + '\n')
            file_epsilon.write(str(epsilon) + '\n')
            file_loss.write(str(loss.numpy()) + '\n')
            file_rewards.flush()
            file_epsilon.flush()
            file_loss.flush()
            
            print(rewards_history[-1])
            env.reset()
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if episode_count % 50 == 0:

        model.compile(optimizer=optimizer, loss=loss_function)
        model_target.compile(optimizer=optimizer, loss=loss_function)
        model.save('train_model.h5')
        model_target.save('target_model.h5')

    if running_reward > 10000000:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

TypeError: 'float' object is not subscriptable

In [10]:
rewards_history[-1]

1.3333767006281225